In [8]:
import numpy as np
from sklearn.datasets import load_iris
from collections import Counter
import random

In [9]:
# Tải dữ liệu Iris
iris = load_iris()
X, y = iris.data, iris.target
feature_names = iris.feature_names

In [10]:
# Hàm tính entropy
def entropy(y):
    counts = np.bincount(y)  # Đếm số lượng mỗi label
    probabilities = counts / len(y)  # Tính xác suất của mỗi label
    return -np.sum([p * np.log2(p) for p in probabilities if p > 0])


# Hàm chia dữ liệu theo điều kiện
def split_dataset(X, y, feature_index, threshold):
    # Kiểm tra từng item trong mảng có thỏa điền kiện, trả về mảng boolean
    left_idx = X[:, feature_index] <= threshold
    right_idx = X[:, feature_index] > threshold
    return X[left_idx], y[left_idx], X[right_idx], y[right_idx]


# Tìm feature và ngưỡng tốt nhất
def best_split(X, y):
    best_gain = 0
    best_feature = None
    best_threshold = None
    current_entropy = entropy(y)

    n_features = X.shape[1]
    for feature_index in range(n_features):
        # chọn ngưỡng là các giá trị khác nhau của feature
        # để chia dữ liệu thành 2 phần
        thresholds = np.unique(X[:, feature_index])
        for threshold in thresholds:
            X_left, y_left, X_right, y_right = split_dataset(
                X, y, feature_index, threshold
            )
            if len(y_left) == 0 or len(y_right) == 0:
                continue
            p_left = len(y_left) / len(y)
            p_right = 1 - p_left
            # độ lợi thông tin là độ giảm entropy
            gain = current_entropy - (
                p_left * entropy(y_left) + p_right * entropy(y_right)
            )
            # chọn cách chia sao cho độ lợi thông tin lớn nhất
            if gain > best_gain:
                best_gain = gain
                best_feature = feature_index
                best_threshold = threshold
    return best_feature, best_threshold

In [11]:
# Node trong cây
class DecisionNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value  # nhãn nếu là node lá


# Xây dựng cây
def build_tree(X, y, depth=0, max_depth=5):
    # Nếu tất cả nhãn giống nhau hoặc độ sâu lớn hơn max_depth
    # thì trả về node lá với nhãn phổ biến nhất
    if len(set(y)) == 1 or depth >= max_depth:
        most_common = Counter(y).most_common(1)[0][0]
        return DecisionNode(value=most_common)

    # Tìm feature và ngưỡng tốt nhất để chia dữ liệu
    feature, threshold = best_split(X, y)

    # Nếu không tìm thấy feature nào để chia thì trả về node lá
    # với nhãn phổ biến nhất
    if feature is None:
        most_common = Counter(y).most_common(1)[0][0]
        return DecisionNode(value=most_common)

    # Chia dữ liệu thành 2 phần theo feature và ngưỡng đã tìm được
    X_left, y_left, X_right, y_right = split_dataset(X, y, feature, threshold)

    # Tiếp tục xây dựng cây cho dữ liệu đã chia
    left = build_tree(X_left, y_left, depth + 1, max_depth)
    right = build_tree(X_right, y_right, depth + 1, max_depth)
    return DecisionNode(feature, threshold, left, right)


# Dự đoán
def predict_one(x, node):
    if node.value is not None:
        return node.value
    if x[node.feature] <= node.threshold:
        return predict_one(x, node.left)
    else:
        return predict_one(x, node.right)


def predict(X, tree):
    return [predict_one(x, tree) for x in X]

In [ ]:
class RandomForest:
    def __init__(self, n_trees=10, max_depth=5, max_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.max_features = max_features
        self.trees = []

    def bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, n_samples, replace=True)
        return X[indices], y[indices]

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            # Bootstrap sampling
            X_sample, y_sample = self.bootstrap_sample(X, y)

            # Select random subset of features if max_features is specified
            if self.max_features:
                feature_indices = random.sample(range(X.shape[1]), self.max_features)
                X_sample = X_sample[:, feature_indices]
            else:
                feature_indices = None

            # Build a decision tree
            tree = build_tree(X_sample, y_sample, max_depth=self.max_depth)
            self.trees.append((tree, feature_indices))

    def predict_one(self, x):
        # Collect predictions from all trees
        predictions = []
        for tree, feature_indices in self.trees:
            if feature_indices:
                x_subset = x[feature_indices]
            else:
                x_subset = x
            predictions.append(predict_one(x_subset, tree))
        # Return the majority vote
        return Counter(predictions).most_common(1)[0][0]

    def predict(self, X):
        return [self.predict_one(x) for x in X]

In [ ]:
# Huấn luyện và đánh giá
tree = build_tree(X, y, max_depth=3)
y_pred = predict(X, tree)
accuracy = np.mean(y_pred == y)
print(f"Accuracy of single decision tree: {accuracy:.2f}")
forest = RandomForest(n_trees=10, max_depth=3, max_features=2)
forest.fit(X, y)
y_pred_forest = forest.predict(X)
accuracy_forest = np.mean(y_pred_forest == y)
print(f"Accuracy of random forest: {accuracy_forest:.2f}")

Accuracy of single decision tree: 0.97
Accuracy of random forest: 0.97
